In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt, timedelta
import winsound
import plotly.graph_objects as go

access_exports_filepath =r"S:\BSA SM Shared\11. Analytics & Reporting Team\11.11 Team Admin\SCDB databases\00_LS_Access_Databases_Price_Errors"
data_exploration_filepath = r"S:\BSA SM Shared\11. Analytics & Reporting Team\11.30 Using Python\01 Python Code\06 Pricing Errors\00 Data Exploration"
price_calculator_filepath =r"S:\BSA SM Shared\11. Analytics & Reporting Team\11.1 BP=SP Project\11.1.92 New Price Calculator Parameters"
output_filepath =r"S:\BSA SM Shared\11. Analytics & Reporting Team\11.30 Using Python\01 Python Code\06 Pricing Errors\00 Data Exploration\Output files T7"

curr_date = dt.today().strftime('%Y%m%d')


## Part 1 - Read Data Files

## a. Legacy Sell Prices

In [2]:
%%time
#3 mins
legacy_df = pd.read_csv(access_exports_filepath+r"/PRODDB_V_PRODDB_PRICE120230206.txt", header = 0)
legacy_df=legacy_df.rename(columns={"B1_PRICE":"B1_SELL_PRICE"})
print(np.shape(legacy_df ))
winsound.Beep(310,500)
winsound.Beep(240,300)  
legacy_df 

(8840243, 13)
CPU times: total: 16 s
Wall time: 1min 26s


,NPC,PG_ID,B1_QTY,B1_SELL_PRICE,B2_QTY,B2_PRICE,B3_QTY,B3_PRICE,B4_QTY,B4_PRICE,B5_QTY,B5_PRICE,EFFECTIVE_DATE
0,FSQ431,1.0,1.0,118.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25/3/2019 00:00:00
1,FSQ429,1.0,1.0,118.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25/3/2019 00:00:00
2,FSQ430,1.0,1.0,120.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25/3/2019 00:00:00
3,FSQ432,1.0,1.0,78.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25/3/2019 00:00:00
4,FSQ433,1.0,1.0,78.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25/3/2019 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8840238,GAZ13860,6957.0,1.0,109.36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30/1/2023 00:00:00
8840239,GAZ13872,6957.0,1.0,392.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30/1/2023 00:00:00
8840240,GAZ13896,6957.0,1.0,392.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30/1/2023 00:00:00
8840241,GAZ13900,6957.0,1.0,392.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30/1/2023 00:00:00


In [3]:
%%time
mapping_folder = r"S:\BSA SM Shared\3. Pricing\3.0 Pricing Regulation\99_Mapping_Files"
product_extract_map = pd.read_csv(mapping_folder+r'\Weekly Pricing Product Extract1.csv')
print(np.shape(product_extract_map))
winsound.Beep(310,500)
winsound.Beep(240,300)  
product_extract_map

(637357, 9)
CPU times: total: 1.25 s
Wall time: 5.13 s


,ITEM_NUMBER,NPC,ITEM_STATUS,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,UNIT_OF_MEASURE,MBMS
0,ABX005,ABX005,Transacted,0.0,Stocked,Category_Tower_01,EA,Each,N
1,ABX008,ABX008,Transacted,0.0,Stocked,Category_Tower_01,EA,Each,N
2,ABX023,ABX023,Transacted,0.0,Stocked,Category_Tower_01,EA,Each,N
3,ABX030,ABX030,Transacted,0.0,Stocked,Category_Tower_01,EA,Each,N
4,ABX035,ABX035,Transacted,20.0,Blue Diamond,Category_Tower_01,EA,Each,N
...,...,...,...,...,...,...,...,...,...
637352,WHJ090,WHJ090,Transacted,20.0,Blue Diamond,Category_Tower_11,RL,Roll,N
637353,WHJ091,WHJ091,Transacted,20.0,Blue Diamond,Category_Tower_11,RL,Roll,N
637354,WHJ092,WHJ092,Transacted,20.0,Blue Diamond,Category_Tower_11,RL,Roll,N
637355,ZZA85000,ZZA85000,Transacted,20.0,e-Direct,Category_Tower_11,EA,Each,N


In [4]:
product_extract_map=product_extract_map.drop(['ITEM_NUMBER','UNIT_OF_MEASURE'], axis=1)
product_extract_map_transacted=product_extract_map.loc[product_extract_map['ITEM_STATUS']=='Transacted']
print(np.shape(product_extract_map_transacted))

(589739, 7)


In [5]:
legacy = pd.merge(legacy_df,product_extract_map,on='NPC',how='left')
print(np.shape(legacy))
T7_legacy_df=legacy.loc[legacy['ITEM_CLASS']=='Category_Tower_07']
print(np.shape(T7_legacy_df))

(8840243, 19)
(1360, 19)


In [6]:
# how many multiband
T7_multiband_band = T7_legacy_df.loc[T7_legacy_df['B2_QTY']!=0]
print(np.shape(T7_multiband_band))
T7_legacy_df=T7_legacy_df.loc[T7_legacy_df['B2_QTY']==0]
T7_legacy_df=T7_legacy_df.drop(['B1_QTY','B2_QTY', 'B2_PRICE',
       'B3_QTY', 'B3_PRICE', 'B4_QTY', 'B4_PRICE', 'B5_QTY', 'B5_PRICE'], axis=1)
T7_legacy_df=T7_legacy_df[T7_legacy_df['ITEM_STATUS']=='Transacted']
T7_legacy_df.loc[:,'B1_EXVAT'] = T7_legacy_df['B1_SELL_PRICE']/\
(T7_legacy_df['PERCENTAGE_RATE']+100)*100
print(np.shape(T7_legacy_df))
T7_legacy_df=T7_legacy_df.drop(['ITEM_STATUS'], axis=1)

(0, 19)
(1351, 11)


## b. Read Contract Buy Prices

In [7]:
%%time
# 20 seconds
# from Sales TransactionsLS.accdb/Contract_line
contract_line = pd.read_csv(access_exports_filepath +r"/Qry4_contract_buy_prices20230206.txt")
contract_line=contract_line.rename(columns={"B1_PRICE":"B1_National_BUY_PRICE"})
contract_line=contract_line.drop(["STATUS_ID","B2_QTY","B2_PRICE","EFFECTIVE_DATE"], axis=1)
winsound.Beep(310,500)
winsound.Beep(240,300)  
contract_line

CPU times: total: 1.12 s
Wall time: 6.62 s


,NPC,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID
0,FVT013,59.790000,22/11/2021 00:00:00,AA135990
1,FVT030,31.820000,22/11/2021 00:00:00,AA135990
2,ENA014,32.510000,14/3/2022 00:00:00,AA137076
3,FKW1791,50.000000,21/2/2018 00:00:00,AA130558
4,FVT046,108.770000,24/12/2021 00:00:00,AA135992
...,...,...,...,...
647749,GAZ86109,1355.000000,1/2/2023 00:00:00,AA133878
647750,GAZ85850,1355.000000,1/2/2023 00:00:00,AA133878
647751,FCB89042,148.450302,24/1/2023 00:00:00,AA135320
647752,FCB88560,173.355336,24/1/2023 00:00:00,AA135320


## c. Read Conlin - Regional Buy Prices

In [8]:
%%time
# 2 mins
# from "00_LS_1_2_Prices_margins_review\Access_Databases\Database_old format_Conlin.mdb/Resus"
# Query1-Conlin_with_contract_num
conlin_df = pd.read_csv(access_exports_filepath+r"/Conlin_with_contract_num20230206.txt", header = 0)
#legacy_df=legacy_df.rename(columns={"B1_PRICE":"B1_SELL_PRICE"})
winsound.Beep(310,500)
winsound.Beep(240,300)  
print(np.shape(conlin_df))
conlin_df

(8183396, 5)
CPU times: total: 5.5 s
Wall time: 28.5 s


,WHSE_CODE,CL_DISTR_NO,CL_NSV_CODE,CL_ORD_PRC_BND_1,CL_CONTR_CODE
0,901,B2,ISY14862,125.00,AA035180
1,901,J7,FFX014,346.50,AA108368
2,901,26,FYU694,120.00,AA116218
3,901,61,FQC153,282.21,AA124248
4,901,C6,FGG912,150.00,AA125992
...,...,...,...,...,...
8183391,917,WV,FRQ1891,4000.00,AA130004
8183392,917,X1,FRQ1891,4000.00,AA130004
8183393,917,2U,FRQ1892,4000.00,AA130004
8183394,917,AG,FRQ1892,4000.00,AA130004


In [9]:
%%time
# 3min 40
pgidmap_df = pd.read_csv(access_exports_filepath+r'\Proddb_v_purchasing_Groups_Npc120230131.txt', header = 0)
print(np.shape(pgidmap_df))
pgidmap_df = pgidmap_df.astype({'WHSE_CODE':'int','PG_ID':'int'})
pgidmap_df.head(3)

(33792061, 4)
CPU times: total: 15.5 s
Wall time: 1min 10s


,NPC,WHSE_CODE,DIST_CODE,PG_ID
0,CFP2127H,907,ID,5568
1,CFP2127H,907,MR,5568
2,CFP2127H,907,ME,5569


In [10]:
%%time
# 25 secs
conlin_regional= pd.merge(conlin_df, pgidmap_df,  how='left', left_on=['WHSE_CODE','CL_DISTR_NO','CL_NSV_CODE'], right_on = ['WHSE_CODE','DIST_CODE','NPC'])
regional = pd.merge(conlin_regional,product_extract_map,on='NPC',how='left')
print(np.shape(regional))
regional

(8183396, 14)
CPU times: total: 28.4 s
Wall time: 28.7 s


,WHSE_CODE,CL_DISTR_NO,CL_NSV_CODE,CL_ORD_PRC_BND_1,CL_CONTR_CODE,NPC,DIST_CODE,PG_ID,ITEM_STATUS,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS
0,901,B2,ISY14862,125.00,AA035180,ISY14862,B2,631.0,Transacted,20.0,e-Direct,Category_Tower_04,EA,Y
1,901,J7,FFX014,346.50,AA108368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,901,26,FYU694,120.00,AA116218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,901,61,FQC153,282.21,AA124248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,901,C6,FGG912,150.00,AA125992,FGG912,C6,2777.0,Transacted,20.0,e-Direct,Category_Tower_06,EA,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8183391,917,WV,FRQ1891,4000.00,AA130004,FRQ1891,WV,2804.0,Transacted,20.0,e-Direct,Category_Tower_06,EA,Y
8183392,917,X1,FRQ1891,4000.00,AA130004,FRQ1891,X1,3072.0,Transacted,20.0,e-Direct,Category_Tower_06,EA,Y
8183393,917,2U,FRQ1892,4000.00,AA130004,FRQ1892,2U,2837.0,Transacted,20.0,e-Direct,Category_Tower_06,EA,Y
8183394,917,AG,FRQ1892,4000.00,AA130004,FRQ1892,AG,4327.0,Transacted,20.0,e-Direct,Category_Tower_06,EA,Y


In [11]:
regional.groupby('ITEM_CLASS').size().sort_values(ascending=False) \
  .reset_index(name='NPC')

,ITEM_CLASS,NPC
0,Category_Tower_04,5301674
1,Category_Tower_06,1667501
2,Category_Tower_01,2279
3,Category_Tower_08,1033
4,Category_Tower_02,757
5,Category_Tower_07,10


In [12]:
T7_regional_df=regional.loc[regional['ITEM_CLASS']=='Category_Tower_07']
print(np.shape(T7_regional_df))
T7_regional_df=T7_regional_df[T7_regional_df['ITEM_STATUS']=='Transacted']
T7_regional_df=T7_regional_df.drop(['ITEM_STATUS'], axis=1)

(10, 14)


In [13]:
print(np.shape(T7_regional_df))

(10, 13)


# Part 2 - Merge Data Files

In [14]:
T7_legacy_df.columns

Index(['NPC', 'PG_ID', 'B1_SELL_PRICE', 'EFFECTIVE_DATE', 'PERCENTAGE_RATE',
       'CHANNEL', 'ITEM_CLASS', 'PRIMARY_UOM_CODE', 'MBMS', 'B1_EXVAT'],
      dtype='object')

In [15]:
contract_line.columns

Index(['NPC', 'B1_National_BUY_PRICE', 'CP_EFFECTIVE_DATE', 'CONTRACT_ID'], dtype='object')

In [16]:
%%time
#merge legacy with contract line
legacy_with_contract = pd.merge(T7_legacy_df,contract_line,on='NPC',how='left')
legacy_with_contract

CPU times: total: 328 ms
Wall time: 324 ms


,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID
0,JMF164,2885.0,25.87,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,21.558333,21.5600,3/2/2017 00:00:00,AA136752
1,JMF165,2868.0,22.05,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,18.375000,18.3800,3/2/2017 00:00:00,AA136752
2,FBE19890,1.0,1234.80,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1029.000000,1029.0000,1/4/2022 00:00:00,AA136764
3,JAH018,1.0,384.00,25/3/2019 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,320.000000,320.0000,7/1/2016 00:00:00,AA136734
4,JAH019,1.0,312.49,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,260.408333,260.4125,1/4/2022 00:00:00,AA136734
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,FBE15951,5569.0,66.05,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,55.041667,54.0000,1/4/2022 00:00:00,AA136762
1347,FBE15953,5569.0,45.25,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,37.708333,37.0000,1/4/2022 00:00:00,AA136762
1348,FBE16158,5569.0,72.16,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,60.133333,59.0000,1/4/2022 00:00:00,AA136762
1349,FBE15668,1918.0,1490.83,3/2/2023 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1242.358333,317.6280,1/4/2022 00:00:00,AA136756


In [17]:
# no contract buy price
not_in_contract=legacy_with_contract.loc[legacy_with_contract['CONTRACT_ID'].isna()]
not_in_contract

,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID


In [18]:
%%time
#merge with regional prices
prices_df = pd.merge(legacy_with_contract, T7_regional_df,  how='outer', left_on=['NPC','PG_ID'],right_on=['CL_NSV_CODE','PG_ID'])
prices_df

CPU times: total: 0 ns
Wall time: 9.9 ms


,NPC_x,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE_x,CHANNEL_x,ITEM_CLASS_x,PRIMARY_UOM_CODE_x,MBMS_x,B1_EXVAT,...,CL_NSV_CODE,CL_ORD_PRC_BND_1,CL_CONTR_CODE,NPC_y,DIST_CODE,PERCENTAGE_RATE_y,CHANNEL_y,ITEM_CLASS_y,PRIMARY_UOM_CODE_y,MBMS_y
0,JMF164,2885.0,25.87,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,21.558333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JMF165,2868.0,22.05,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,18.375000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FBE19890,1.0,1234.80,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1029.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JAH018,1.0,384.00,25/3/2019 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,320.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,JAH019,1.0,312.49,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,260.408333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,FBE15951,5569.0,66.05,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,55.041667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1351,FBE15953,5569.0,45.25,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,37.708333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1352,FBE16158,5569.0,72.16,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,60.133333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1353,FBE15668,1918.0,1490.83,3/2/2023 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1242.358333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
prices_df.to_csv(output_filepath+r'/T7_Prices_'+curr_date+'.csv', index = False)

# Part 3 - Analysis

In [20]:
prices_df = pd.read_csv(output_filepath+r'/T7_Prices_'+curr_date+'.csv', header = 0)
prices_df.head(5)

,NPC_x,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE_x,CHANNEL_x,ITEM_CLASS_x,PRIMARY_UOM_CODE_x,MBMS_x,B1_EXVAT,...,CL_NSV_CODE,CL_ORD_PRC_BND_1,CL_CONTR_CODE,NPC_y,DIST_CODE,PERCENTAGE_RATE_y,CHANNEL_y,ITEM_CLASS_y,PRIMARY_UOM_CODE_y,MBMS_y
0,JMF164,2885.0,25.87,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,21.558333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JMF165,2868.0,22.05,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,18.375000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FBE19890,1.0,1234.80,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1029.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JAH018,1.0,384.00,25/3/2019 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,320.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,JAH019,1.0,312.49,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,260.408333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## A. Split out National buy only

In [21]:
# no contract buy price
national_buy=prices_df.loc[prices_df['CL_CONTR_CODE'].isna()]
national_buy

,NPC_x,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE_x,CHANNEL_x,ITEM_CLASS_x,PRIMARY_UOM_CODE_x,MBMS_x,B1_EXVAT,...,CL_NSV_CODE,CL_ORD_PRC_BND_1,CL_CONTR_CODE,NPC_y,DIST_CODE,PERCENTAGE_RATE_y,CHANNEL_y,ITEM_CLASS_y,PRIMARY_UOM_CODE_y,MBMS_y
0,JMF164,2885.0,25.87,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,21.558333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JMF165,2868.0,22.05,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,18.375000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FBE19890,1.0,1234.80,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1029.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JAH018,1.0,384.00,25/3/2019 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,320.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,JAH019,1.0,312.49,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,260.408333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,FBE15951,5569.0,66.05,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,55.041667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1351,FBE15953,5569.0,45.25,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,37.708333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1352,FBE16158,5569.0,72.16,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,60.133333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1353,FBE15668,1918.0,1490.83,3/2/2023 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1242.358333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
regional_buy=prices_df.loc[prices_df['CL_CONTR_CODE'].notnull()]
regional_buy

,NPC_x,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE_x,CHANNEL_x,ITEM_CLASS_x,PRIMARY_UOM_CODE_x,MBMS_x,B1_EXVAT,...,CL_NSV_CODE,CL_ORD_PRC_BND_1,CL_CONTR_CODE,NPC_y,DIST_CODE,PERCENTAGE_RATE_y,CHANNEL_y,ITEM_CLASS_y,PRIMARY_UOM_CODE_y,MBMS_y
1333,FBE15668,2489.0,397.04,24/12/2021 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,330.866667,...,FBE15668,330.87,AA131360,FBE15668,C6,20.0,e-Direct,Category_Tower_07,EA,N
1334,FBE15668,2489.0,397.04,24/12/2021 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,330.866667,...,FBE15668,330.87,AA131360,FBE15668,PA,20.0,e-Direct,Category_Tower_07,EA,N
1335,FBE18728,2489.0,561.81,24/12/2021 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,468.175000,...,FBE18728,468.18,AA131360,FBE18728,C6,20.0,e-Direct,Category_Tower_07,EA,N
1336,FBE18728,2489.0,561.81,24/12/2021 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,468.175000,...,FBE18728,468.18,AA131360,FBE18728,PA,20.0,e-Direct,Category_Tower_07,EA,N
1340,FBE15668,407.0,397.03,30/9/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,330.858333,...,FBE15668,330.86,AA131360,FBE15668,TK,20.0,e-Direct,Category_Tower_07,EA,N
1341,FBE15668,407.0,397.03,30/9/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,330.858333,...,FBE15668,330.86,AA131360,FBE15668,ZL,20.0,e-Direct,Category_Tower_07,EA,N
1342,FBE18728,407.0,561.81,30/9/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,468.175000,...,FBE18728,468.18,AA131360,FBE18728,TK,20.0,e-Direct,Category_Tower_07,EA,N
1343,FBE18728,407.0,561.81,30/9/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,468.175000,...,FBE18728,468.18,AA131360,FBE18728,ZL,20.0,e-Direct,Category_Tower_07,EA,N
1344,FBE15668,1459.0,397.03,30/9/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,330.858333,...,FBE15668,330.86,AA131360,FBE15668,8U,20.0,e-Direct,Category_Tower_07,EA,N
1345,FBE18728,1459.0,561.81,30/9/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,468.175000,...,FBE18728,468.18,AA131360,FBE18728,8U,20.0,e-Direct,Category_Tower_07,EA,N


In [23]:
regional_buy.to_csv(output_filepath+r'/T7_regional_buy_'+curr_date+'.csv', index = False)

## Add Margin

In [24]:
national_buy['%Margin']=round(((national_buy['B1_EXVAT']-national_buy['B1_National_BUY_PRICE'])/national_buy['B1_EXVAT']*100),3)
national_buy=national_buy.drop(['WHSE_CODE', 'CL_DISTR_NO', 'CL_NSV_CODE',
       'CL_ORD_PRC_BND_1', 'CL_CONTR_CODE', 'NPC_y', 'DIST_CODE',
       'PERCENTAGE_RATE_y', 'CHANNEL_y', 'ITEM_CLASS_y', 'PRIMARY_UOM_CODE_y',
       'MBMS_y'], axis=1)
national_buy.columns = national_buy.columns.str.replace("_x", "")
national_buy

C:\Users\syed.imam\AppData\Local\Temp\ipykernel_109476\1085324364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  national_buy['%Margin']=round(((national_buy['B1_EXVAT']-national_buy['B1_National_BUY_PRICE'])/national_buy['B1_EXVAT']*100),3)


,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID,%Margin
0,JMF164,2885.0,25.87,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,21.558333,21.5600,3/2/2017 00:00:00,AA136752,-0.008
1,JMF165,2868.0,22.05,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,18.375000,18.3800,3/2/2017 00:00:00,AA136752,-0.027
2,FBE19890,1.0,1234.80,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1029.000000,1029.0000,1/4/2022 00:00:00,AA136764,0.000
3,JAH018,1.0,384.00,25/3/2019 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,320.000000,320.0000,7/1/2016 00:00:00,AA136734,0.000
4,JAH019,1.0,312.49,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,260.408333,260.4125,1/4/2022 00:00:00,AA136734,-0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,FBE15951,5569.0,66.05,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,55.041667,54.0000,1/4/2022 00:00:00,AA136762,1.893
1351,FBE15953,5569.0,45.25,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,37.708333,37.0000,1/4/2022 00:00:00,AA136762,1.878
1352,FBE16158,5569.0,72.16,5/10/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,60.133333,59.0000,1/4/2022 00:00:00,AA136762,1.885
1353,FBE15668,1918.0,1490.83,3/2/2023 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1242.358333,317.6280,1/4/2022 00:00:00,AA136756,74.433


In [25]:
national_buy.to_csv(output_filepath+r'/T7_national_buy_'+curr_date+'.csv', index = False)

## Read National buy

In [26]:
national_buy = pd.read_csv(output_filepath+r'/T7_national_buy_'+curr_date+'.csv', header = 0)
national_buy.head(5)

,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID,%Margin
0,JMF164,2885.0,25.87,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,21.558333,21.5600,3/2/2017 00:00:00,AA136752,-0.008
1,JMF165,2868.0,22.05,27/5/2019 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,18.375000,18.3800,3/2/2017 00:00:00,AA136752,-0.027
2,FBE19890,1.0,1234.80,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,EA,N,1029.000000,1029.0000,1/4/2022 00:00:00,AA136764,0.000
3,JAH018,1.0,384.00,25/3/2019 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,320.000000,320.0000,7/1/2016 00:00:00,AA136734,0.000
4,JAH019,1.0,312.49,1/4/2022 00:00:00,20.0,e-Direct,Category_Tower_07,BX,N,260.408333,260.4125,1/4/2022 00:00:00,AA136734,-0.002


In [27]:
national_buy['EFFECTIVE_DATE'] =  pd.to_datetime(national_buy['EFFECTIVE_DATE'])
national_buy['CP_EFFECTIVE_DATE'] =  pd.to_datetime(national_buy['CP_EFFECTIVE_DATE'])
national_buy.sort_values(by=['B1_SELL_PRICE'],ascending=False)
national_buy

,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID,%Margin
0,JMF164,2885.0,25.87,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,21.558333,21.5600,2017-03-02,AA136752,-0.008
1,JMF165,2868.0,22.05,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,18.375000,18.3800,2017-03-02,AA136752,-0.027
2,FBE19890,1.0,1234.80,2022-01-04,20.0,e-Direct,Category_Tower_07,EA,N,1029.000000,1029.0000,2022-01-04,AA136764,0.000
3,JAH018,1.0,384.00,2019-03-25,20.0,e-Direct,Category_Tower_07,BX,N,320.000000,320.0000,2016-07-01,AA136734,0.000
4,JAH019,1.0,312.49,2022-01-04,20.0,e-Direct,Category_Tower_07,BX,N,260.408333,260.4125,2022-01-04,AA136734,-0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340,FBE15951,5569.0,66.05,2022-05-10,20.0,e-Direct,Category_Tower_07,EA,N,55.041667,54.0000,2022-01-04,AA136762,1.893
1341,FBE15953,5569.0,45.25,2022-05-10,20.0,e-Direct,Category_Tower_07,EA,N,37.708333,37.0000,2022-01-04,AA136762,1.878
1342,FBE16158,5569.0,72.16,2022-05-10,20.0,e-Direct,Category_Tower_07,EA,N,60.133333,59.0000,2022-01-04,AA136762,1.885
1343,FBE15668,1918.0,1490.83,2023-03-02,20.0,e-Direct,Category_Tower_07,EA,N,1242.358333,317.6280,2022-01-04,AA136756,74.433


In [28]:
zero_buy=national_buy.loc[national_buy['B1_National_BUY_PRICE']==0]
np.shape(zero_buy)

(0, 14)

In [29]:
zero_sell=national_buy.loc[national_buy['B1_SELL_PRICE']==0]
np.shape(zero_sell)

(0, 14)

In [30]:
nullmargin=national_buy.loc[national_buy['%Margin'].isna()]
print(np.shape(nullmargin))
nullmargin.sort_values(by=['NPC','PG_ID'],ascending=True)

(0, 14)


,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID,%Margin


In [31]:
df=national_buy.loc[national_buy['%Margin'].notnull()]
print(np.shape(df))
df=df.loc[df['B1_SELL_PRICE']>0.01]
print(np.shape(df))
df=df.loc[df['B1_National_BUY_PRICE']>0.01]
print(np.shape(df))

(1345, 14)
(1345, 14)
(1345, 14)


In [32]:
national_buy_margin = national_buy.loc[national_buy['%Margin'].notnull()]
national_buy_margin.describe()

,PG_ID,B1_SELL_PRICE,PERCENTAGE_RATE,B1_EXVAT,B1_National_BUY_PRICE,%Margin
count,1345.000000,1345.000000,1345.0,1345.000000,1345.000000,1345.000000
mean,3706.919703,582.428082,20.0,485.356735,481.460323,0.869493
std,2532.605029,1541.678314,0.0,1284.731928,1284.438499,13.380543
min,1.000000,14.450000,20.0,12.041667,13.000000,-473.160000
25%,1.000000,91.040000,20.0,75.866667,75.000000,0.000000
50%,5568.000000,236.100000,20.0,196.750000,195.000000,0.386000
75%,5570.000000,465.870000,20.0,388.225000,378.000000,1.895000
max,6859.000000,9600.000000,20.0,8000.000000,8000.000000,74.433000


In [33]:
df.sort_values(by=['%Margin'],ascending=False)

,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID,%Margin
1343,FBE15668,1918.0,1490.83,2023-03-02,20.0,e-Direct,Category_Tower_07,EA,N,1242.358333,317.6280,2022-01-04,AA136756,74.433
294,JAH020,6606.0,652.71,2022-07-28,20.0,e-Direct,Category_Tower_07,BX,N,543.925000,271.9632,2022-01-04,AA136734,50.000
295,JAH022,6606.0,481.79,2022-07-28,20.0,e-Direct,Category_Tower_07,BX,N,401.491667,200.7489,2022-01-04,AA136734,49.999
1178,JAH022,4375.0,334.58,2022-01-04,20.0,e-Direct,Category_Tower_07,BX,N,278.816667,200.7489,2022-01-04,AA136734,28.000
310,JAH020,4375.0,453.27,2022-01-04,20.0,e-Direct,Category_Tower_07,BX,N,377.725000,271.9632,2022-01-04,AA136734,28.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,JAH418,4370.0,97.14,2021-11-29,20.0,e-Direct,Category_Tower_07,BX,N,80.950000,81.8800,2022-01-04,AA136734,-1.149
1333,FBE18195,5571.0,14.45,2022-11-02,20.0,e-Direct,Category_Tower_07,EA,N,12.041667,13.0000,2022-01-04,AA136762,-7.958
416,JMF127,2722.0,19.20,2019-03-29,20.0,e-Direct,Category_Tower_07,EA,N,16.000000,17.5900,2022-03-05,AA137668,-9.938
1332,JAH199,5571.0,251.76,2021-11-29,20.0,e-Direct,Category_Tower_07,BX,N,209.800000,270.0000,2022-01-04,AA136736,-28.694


# Part 4 - National buy only

In [34]:
cf_ncfmap= pd.read_excel(data_exploration_filepath+ r"/PGID_CF_map_20230214.xlsx")
cf_ncfmap

,PG_ID,CENTRALLY_FUNDED,PG_NAME,Category,YTD_LIBRA
0,1,NaN,National,NaN,NaN
1,4,1.0,Herts Supply Management Confederation,Bespoke deal,4111.84
2,5,NaN,Hampshire and Isle of Wight Supplies Confedera...,NaN,NaN
3,6,1.0,Peninsula Purchasing & Supply Alliance,NaN,17104.18
4,10,1.0,Durham and Tees Valley,Bespoke deal,1726.96
...,...,...,...,...,...
5995,7071,NaN,CT01 CF Zoll NPM Worcestershire Ac,NPM,0.00
5996,7072,NaN,CT06 CF MBMS Warrington and Halton Hospitals,NaN,0.00
5997,7073,NaN,CT01 CF NPM Owen Mumford Tier 1,NPM,0.00
5998,7074,NaN,CT01 CF NPM Owen Mumford Tier 2,NPM,0.00


## Margin at PGID and Contract level

In [35]:
non_PGID1=df.loc[df['PG_ID']!=1]
np.shape(non_PGID1)

(966, 14)

In [36]:
medians_contract=non_PGID1.groupby(['CONTRACT_ID', 'PG_ID','CHANNEL'])['%Margin'].median().reset_index(name='nat_median')
medians_contract.sort_values(by=['nat_median'],ascending=True)

,CONTRACT_ID,PG_ID,CHANNEL,nat_median
67,AA136756,1918.0,e-Direct,-199.3635
23,AA136736,5571.0,e-Direct,-28.6940
83,AA136762,5571.0,e-Direct,-7.9580
108,AA137668,3144.0,e-Direct,-0.0150
46,AA136752,1515.0,e-Direct,0.0000
...,...,...,...,...
32,AA136742,4378.0,e-Direct,6.5000
27,AA136738,5375.0,e-Direct,10.2385
26,AA136738,4378.0,e-Direct,10.2385
7,AA136734,4375.0,e-Direct,28.0000


In [37]:
st_dev_contract=non_PGID1.groupby(['CONTRACT_ID', 'PG_ID','CHANNEL'])['%Margin'].std(ddof=0).reset_index(name='st_dev')
st_dev_contract.sort_values(by=['st_dev'],ascending=True)

,CONTRACT_ID,PG_ID,CHANNEL,st_dev
0,AA132392,1786.0,e-Direct,0.00000
38,AA136744,4056.0,e-Direct,0.00000
40,AA136746,4056.0,e-Direct,0.00000
46,AA136752,1515.0,e-Direct,0.00000
49,AA136752,2867.0,e-Direct,0.00000
...,...,...,...,...
31,AA136738,5836.0,e-Direct,3.96200
107,AA137668,2722.0,e-Direct,4.30328
27,AA136738,5375.0,e-Direct,7.23850
26,AA136738,4378.0,e-Direct,7.23850


In [38]:
means_contract=non_PGID1.groupby(['CONTRACT_ID', 'PG_ID','CHANNEL'])['%Margin'].mean().reset_index(name='Mean')
means_contract.sort_values(by=['Mean'],ascending=True)

,CONTRACT_ID,PG_ID,CHANNEL,Mean
67,AA136756,1918.0,e-Direct,-199.3635
23,AA136736,5571.0,e-Direct,-28.6940
83,AA136762,5571.0,e-Direct,-7.9580
107,AA137668,2722.0,e-Direct,-2.4845
108,AA137668,3144.0,e-Direct,-0.0150
...,...,...,...,...
32,AA136742,4378.0,e-Direct,6.5000
26,AA136738,4378.0,e-Direct,10.2385
27,AA136738,5375.0,e-Direct,10.2385
7,AA136734,4375.0,e-Direct,28.0000


In [39]:
def channel_margin (row):
    if row['CHANNEL'] == 'e-Direct' :
      return 3
    if row['CHANNEL'] == 'Blue Diamond' :
      return 5
    return 10

In [40]:
medians_std_contract= pd.merge(medians_contract, st_dev_contract,  how='left', on=['PG_ID','CONTRACT_ID','CHANNEL'])
medians_std_means_contract= pd.merge(medians_std_contract, means_contract,  how='left', on=['PG_ID','CONTRACT_ID','CHANNEL'])
T7_margin_stats_contract= pd.merge(medians_std_means_contract, cf_ncfmap,  how='left', on=['PG_ID'])
T7_margin_stats_contract['Channel_Margin'] = T7_margin_stats_contract.apply (lambda row: channel_margin(row), axis=1)
T7_margin_stats_contract['Margin Median Difference']=round((T7_margin_stats_contract['nat_median']-T7_margin_stats_contract['Channel_Margin']),3)
cols = ['CONTRACT_ID', 'PG_ID',  'PG_NAME', 'Category','CENTRALLY_FUNDED','CHANNEL', 'nat_median', 'st_dev', 'Mean','YTD_LIBRA','Channel_Margin','Margin Median Difference']
T7_margin_stats_contract = T7_margin_stats_contract[cols]
T7_margin_stats_contract.to_excel(output_filepath+r'/T7_margin_stats_contract_'+curr_date+'.xlsx', index = False) 

In [41]:
T7_margin_stats_contract.apply (lambda row: channel_margin(row), axis=1)

0      3
1      3
2      3
3      3
4      3
      ..
109    3
110    3
111    3
112    3
113    3
Length: 114, dtype: int64

In [42]:
T7_margin_stats_contract

,CONTRACT_ID,PG_ID,PG_NAME,Category,CENTRALLY_FUNDED,CHANNEL,nat_median,st_dev,Mean,YTD_LIBRA,Channel_Margin,Margin Median Difference
0,AA132392,1786.0,Northern Ireland Core Catalogue,Core,0.0,e-Direct,2.5000,0.000000,2.5000,3294.44,3,-0.500
1,AA136728,6006.0,Selective Internal Radiation Therapy (HCTED),NPM,1.0,e-Direct,0.0000,0.000000,0.0000,38400.00,3,-3.000
2,AA136730,6006.0,Selective Internal Radiation Therapy (HCTED),NPM,1.0,e-Direct,0.0000,0.000000,0.0000,38400.00,3,-3.000
3,AA136734,190.0,Ministry of Defence,Core,0.0,e-Direct,5.4985,2.501500,5.4985,1630.56,3,2.498
4,AA136734,2104.0,Welsh Health Boards,Core,0.0,e-Direct,2.9980,0.002291,2.9975,4844.38,3,-0.002
...,...,...,...,...,...,...,...,...,...,...,...,...
109,AA137668,3449.0,The Christie Nhs Foundation Trust,NaN,1.0,e-Direct,0.0000,0.000000,0.0000,7644.00,3,-3.000
110,AA137668,5568.0,ICF NCF Band 1 0-24.9,Core,0.0,e-Direct,2.5670,0.000000,2.5670,27.60,3,-0.433
111,AA137668,5569.0,ICF NCF Band 2 25-49.9,Core,0.0,e-Direct,1.8610,0.000000,1.8610,115.88,3,-1.139
112,AA137668,5570.0,ICF NCF Band 3 50-74.9,Core,0.0,e-Direct,1.1330,0.000000,1.1330,8.54,3,-1.867


In [43]:
contract_margin = pd.read_excel(price_calculator_filepath+r"/02.Margin By Contract.xlsx",header = 1)
contract_margin.head(5)

,CONTRACT_ID,PG_ID,SUPPLY_ROUTE,PERCENTAGE_MARGIN
0,AA111986,1538,Edirect,2.3
1,AA111990,1151,Edirect,1.6
2,AA111990,1935,Edirect,5.3
3,AA112374,55,Stocked,5.0
4,AA112374,1815,Stocked,6.4


In [44]:
pgid_route_margin = pd.read_excel(price_calculator_filepath+r'/03.Margin by PGID and Route.xlsx', header =1)
pgid_route_margin.head(5)

,PG_ID,SUPPLY_ROUTE,PERCENTAGE_MARGIN
0,-9898,Edirect,3.0
1,-9898,BlueDiamond,5.0
2,-9898,Stocked,10.0
3,50,Edirect,2.6
4,55,Stocked,5.0


In [45]:
contract_pgid_margin= pd.merge(T7_margin_stats_contract, contract_margin,  how='left', left_on=['CONTRACT_ID','PG_ID','CHANNEL'],right_on=['CONTRACT_ID','PG_ID','SUPPLY_ROUTE'])
contract_pgid_margin

,CONTRACT_ID,PG_ID,PG_NAME,Category,CENTRALLY_FUNDED,CHANNEL,nat_median,st_dev,Mean,YTD_LIBRA,Channel_Margin,Margin Median Difference,SUPPLY_ROUTE,PERCENTAGE_MARGIN
0,AA132392,1786.0,Northern Ireland Core Catalogue,Core,0.0,e-Direct,2.5000,0.000000,2.5000,3294.44,3,-0.500,NaN,NaN
1,AA136728,6006.0,Selective Internal Radiation Therapy (HCTED),NPM,1.0,e-Direct,0.0000,0.000000,0.0000,38400.00,3,-3.000,NaN,NaN
2,AA136730,6006.0,Selective Internal Radiation Therapy (HCTED),NPM,1.0,e-Direct,0.0000,0.000000,0.0000,38400.00,3,-3.000,NaN,NaN
3,AA136734,190.0,Ministry of Defence,Core,0.0,e-Direct,5.4985,2.501500,5.4985,1630.56,3,2.498,NaN,NaN
4,AA136734,2104.0,Welsh Health Boards,Core,0.0,e-Direct,2.9980,0.002291,2.9975,4844.38,3,-0.002,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,AA137668,3449.0,The Christie Nhs Foundation Trust,NaN,1.0,e-Direct,0.0000,0.000000,0.0000,7644.00,3,-3.000,NaN,NaN
110,AA137668,5568.0,ICF NCF Band 1 0-24.9,Core,0.0,e-Direct,2.5670,0.000000,2.5670,27.60,3,-0.433,NaN,NaN
111,AA137668,5569.0,ICF NCF Band 2 25-49.9,Core,0.0,e-Direct,1.8610,0.000000,1.8610,115.88,3,-1.139,NaN,NaN
112,AA137668,5570.0,ICF NCF Band 3 50-74.9,Core,0.0,e-Direct,1.1330,0.000000,1.1330,8.54,3,-1.867,NaN,NaN


In [46]:
contract_pgid_route_margin= pd.merge(contract_pgid_margin, pgid_route_margin, how='left', left_on=['PG_ID','CHANNEL'],right_on=['PG_ID','SUPPLY_ROUTE'])
contract_pgid_route_margin

,CONTRACT_ID,PG_ID,PG_NAME,Category,CENTRALLY_FUNDED,CHANNEL,nat_median,st_dev,Mean,YTD_LIBRA,Channel_Margin,Margin Median Difference,SUPPLY_ROUTE_x,PERCENTAGE_MARGIN_x,SUPPLY_ROUTE_y,PERCENTAGE_MARGIN_y
0,AA132392,1786.0,Northern Ireland Core Catalogue,Core,0.0,e-Direct,2.5000,0.000000,2.5000,3294.44,3,-0.500,NaN,NaN,NaN,NaN
1,AA136728,6006.0,Selective Internal Radiation Therapy (HCTED),NPM,1.0,e-Direct,0.0000,0.000000,0.0000,38400.00,3,-3.000,NaN,NaN,NaN,NaN
2,AA136730,6006.0,Selective Internal Radiation Therapy (HCTED),NPM,1.0,e-Direct,0.0000,0.000000,0.0000,38400.00,3,-3.000,NaN,NaN,NaN,NaN
3,AA136734,190.0,Ministry of Defence,Core,0.0,e-Direct,5.4985,2.501500,5.4985,1630.56,3,2.498,NaN,NaN,NaN,NaN
4,AA136734,2104.0,Welsh Health Boards,Core,0.0,e-Direct,2.9980,0.002291,2.9975,4844.38,3,-0.002,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,AA137668,3449.0,The Christie Nhs Foundation Trust,NaN,1.0,e-Direct,0.0000,0.000000,0.0000,7644.00,3,-3.000,NaN,NaN,NaN,NaN
110,AA137668,5568.0,ICF NCF Band 1 0-24.9,Core,0.0,e-Direct,2.5670,0.000000,2.5670,27.60,3,-0.433,NaN,NaN,NaN,NaN
111,AA137668,5569.0,ICF NCF Band 2 25-49.9,Core,0.0,e-Direct,1.8610,0.000000,1.8610,115.88,3,-1.139,NaN,NaN,NaN,NaN
112,AA137668,5570.0,ICF NCF Band 3 50-74.9,Core,0.0,e-Direct,1.1330,0.000000,1.1330,8.54,3,-1.867,NaN,NaN,NaN,NaN


### Writing file on to data exploration folder

In [47]:
contract_pgid_route_margin.to_excel(output_filepath+r'/T7_PGID_margin_stats_with_route_contract_'+curr_date+'.xlsx', index = False,) 

### Filtering out outliers

filter = libra sales not zero or blank
catergory = only blanks
remove duplicates 

In [48]:
contract_pgid_route_margin_outlier = contract_pgid_route_margin.loc[contract_pgid_route_margin['YTD_LIBRA']!=0]

contract_pgid_route_margin_outlier = contract_pgid_route_margin_outlier.loc[contract_pgid_route_margin_outlier['YTD_LIBRA'].notnull()]

contract_pgid_route_margin_outlier = contract_pgid_route_margin_outlier.loc[contract_pgid_route_margin_outlier['Category'].isnull()]

contract_pgid_route_margin_outlier

,CONTRACT_ID,PG_ID,PG_NAME,Category,CENTRALLY_FUNDED,CHANNEL,nat_median,st_dev,Mean,YTD_LIBRA,Channel_Margin,Margin Median Difference,SUPPLY_ROUTE_x,PERCENTAGE_MARGIN_x,SUPPLY_ROUTE_y,PERCENTAGE_MARGIN_y
37,AA136744,3449.0,The Christie Nhs Foundation Trust,NaN,1.0,e-Direct,0.0,0.000000,0.000000,7644.00,3,-3.0,NaN,NaN,NaN,NaN
49,AA136752,2867.0,Oxford Brachytheraphy,NaN,1.0,e-Direct,0.0,0.000000,0.000000,4514.40,3,-3.0,NaN,NaN,NaN,NaN
51,AA136752,2885.0,Guys And St Thomas Trust Brachytherapy Seeds,NaN,1.0,e-Direct,0.0,0.003200,-0.001600,12418.56,3,-3.0,NaN,NaN,NaN,NaN
54,AA136752,3432.0,Radiotherapy - United Lincolnshire Hospitals N...,NaN,1.0,e-Direct,0.0,0.001863,-0.000833,32863.32,3,-3.0,NaN,NaN,NaN,NaN
106,AA137668,1770.0,Poole Hospital Nhs Foundation Trust,NaN,1.0,e-Direct,0.0,0.000000,0.000000,9.98,3,-3.0,NaN,NaN,NaN,NaN
107,AA137668,2722.0,Barts Health Nhs Trust - Brachytherapy,NaN,1.0,e-Direct,0.0,4.303280,-2.484500,12096.00,3,-3.0,NaN,NaN,NaN,NaN
109,AA137668,3449.0,The Christie Nhs Foundation Trust,NaN,1.0,e-Direct,0.0,0.000000,0.000000,7644.00,3,-3.0,NaN,NaN,NaN,NaN


In [49]:
contract_pgid_route_margin_outlier.to_excel(output_filepath+r'/T7_PGID_margin_stats_with_route_contract_'+curr_date+'.xlsx', index = False, sheet_name ='Sheet2') 

In [50]:
contract_pgid_route_margin_outlier.columns

Index(['CONTRACT_ID', 'PG_ID', 'PG_NAME', 'Category', 'CENTRALLY_FUNDED',
       'CHANNEL', 'nat_median', 'st_dev', 'Mean', 'YTD_LIBRA',
       'Channel_Margin', 'Margin Median Difference', 'SUPPLY_ROUTE_x',
       'PERCENTAGE_MARGIN_x', 'SUPPLY_ROUTE_y', 'PERCENTAGE_MARGIN_y'],
      dtype='object')

In [51]:
contract_pgids=contract_pgid_route_margin_outlier.drop(['PG_NAME', 'Category', 'CENTRALLY_FUNDED',
       'CHANNEL', 'nat_median', 'st_dev', 'Mean', 'YTD_LIBRA',
       'Channel_Margin', 'Margin Median Difference', 'SUPPLY_ROUTE_x',
       'PERCENTAGE_MARGIN_x', 'SUPPLY_ROUTE_y', 'PERCENTAGE_MARGIN_y'], axis=1)
contract_pgids

,CONTRACT_ID,PG_ID
37,AA136744,3449.0
49,AA136752,2867.0
51,AA136752,2885.0
54,AA136752,3432.0
106,AA137668,1770.0
107,AA137668,2722.0
109,AA137668,3449.0


In [52]:
contract_pgids = contract_pgids.drop_duplicates(subset=['PG_ID',"CONTRACT_ID"])
contract_pgids

,CONTRACT_ID,PG_ID
37,AA136744,3449.0
49,AA136752,2867.0
51,AA136752,2885.0
54,AA136752,3432.0
106,AA137668,1770.0
107,AA137668,2722.0
109,AA137668,3449.0


In [53]:
## TODO remove 6957
contract_pgid_without_6957 = contract_pgids[contract_pgids['PG_ID'] != 6957]
contract_pgid_without_6957

,CONTRACT_ID,PG_ID
37,AA136744,3449.0
49,AA136752,2867.0
51,AA136752,2885.0
54,AA136752,3432.0
106,AA137668,1770.0
107,AA137668,2722.0
109,AA137668,3449.0


In [54]:
df

,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID,%Margin
0,JMF164,2885.0,25.87,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,21.558333,21.5600,2017-03-02,AA136752,-0.008
1,JMF165,2868.0,22.05,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,18.375000,18.3800,2017-03-02,AA136752,-0.027
2,FBE19890,1.0,1234.80,2022-01-04,20.0,e-Direct,Category_Tower_07,EA,N,1029.000000,1029.0000,2022-01-04,AA136764,0.000
3,JAH018,1.0,384.00,2019-03-25,20.0,e-Direct,Category_Tower_07,BX,N,320.000000,320.0000,2016-07-01,AA136734,0.000
4,JAH019,1.0,312.49,2022-01-04,20.0,e-Direct,Category_Tower_07,BX,N,260.408333,260.4125,2022-01-04,AA136734,-0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340,FBE15951,5569.0,66.05,2022-05-10,20.0,e-Direct,Category_Tower_07,EA,N,55.041667,54.0000,2022-01-04,AA136762,1.893
1341,FBE15953,5569.0,45.25,2022-05-10,20.0,e-Direct,Category_Tower_07,EA,N,37.708333,37.0000,2022-01-04,AA136762,1.878
1342,FBE16158,5569.0,72.16,2022-05-10,20.0,e-Direct,Category_Tower_07,EA,N,60.133333,59.0000,2022-01-04,AA136762,1.885
1343,FBE15668,1918.0,1490.83,2023-03-02,20.0,e-Direct,Category_Tower_07,EA,N,1242.358333,317.6280,2022-01-04,AA136756,74.433


In [55]:
margin_outliers = pd.merge(contract_pgid_without_6957, df,  how='left', on=['PG_ID','CONTRACT_ID'])
margin_outliers

,CONTRACT_ID,PG_ID,NPC,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,%Margin
0,AA136744,3449.0,JNZ038,175.50,2019-03-29,20.0,e-Direct,Category_Tower_07,BX,N,146.250000,146.25,2018-04-26,0.000
1,AA136744,3449.0,JNZ039,269.10,2019-03-29,20.0,e-Direct,Category_Tower_07,BX,N,224.250000,224.25,2018-04-26,0.000
2,AA136752,2867.0,JMF166,23.40,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,19.500000,19.50,2017-06-02,0.000
3,AA136752,2867.0,JMF150,237.60,2019-03-29,20.0,e-Direct,Category_Tower_07,EA,N,198.000000,198.00,2018-01-05,0.000
4,AA136752,2867.0,JMF151,356.40,2019-03-29,20.0,e-Direct,Category_Tower_07,EA,N,297.000000,297.00,2018-01-05,0.000
5,AA136752,2867.0,JMF152,237.60,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,198.000000,198.00,2018-01-05,0.000
6,AA136752,2867.0,JMF153,475.20,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,396.000000,396.00,2018-01-05,0.000
7,AA136752,2867.0,JMF154,356.40,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,297.000000,297.00,2018-01-05,0.000
8,AA136752,2867.0,JMF155,23.76,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,19.800000,19.80,2018-01-05,0.000
9,AA136752,2885.0,JMF164,25.87,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,21.558333,21.56,2017-03-02,-0.008


In [56]:
margin_outliers.to_excel(output_filepath+r'/T7_margin_outlierx_'+curr_date+'.xlsx', index = False) 

### Merging with PV&CC margins

In [57]:
pvcc_margin = pd.merge(margin_outliers,contract_pgid_route_margin, how='left', on=['PG_ID','CONTRACT_ID'])
pvcc_margin


,CONTRACT_ID,PG_ID,NPC,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL_x,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,...,nat_median,st_dev,Mean,YTD_LIBRA,Channel_Margin,Margin Median Difference,SUPPLY_ROUTE_x,PERCENTAGE_MARGIN_x,SUPPLY_ROUTE_y,PERCENTAGE_MARGIN_y
0,AA136744,3449.0,JNZ038,175.50,2019-03-29,20.0,e-Direct,Category_Tower_07,BX,N,...,0.0,0.000000,0.000000,7644.00,3,-3.0,NaN,NaN,NaN,NaN
1,AA136744,3449.0,JNZ039,269.10,2019-03-29,20.0,e-Direct,Category_Tower_07,BX,N,...,0.0,0.000000,0.000000,7644.00,3,-3.0,NaN,NaN,NaN,NaN
2,AA136752,2867.0,JMF166,23.40,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,...,0.0,0.000000,0.000000,4514.40,3,-3.0,NaN,NaN,NaN,NaN
3,AA136752,2867.0,JMF150,237.60,2019-03-29,20.0,e-Direct,Category_Tower_07,EA,N,...,0.0,0.000000,0.000000,4514.40,3,-3.0,NaN,NaN,NaN,NaN
4,AA136752,2867.0,JMF151,356.40,2019-03-29,20.0,e-Direct,Category_Tower_07,EA,N,...,0.0,0.000000,0.000000,4514.40,3,-3.0,NaN,NaN,NaN,NaN
5,AA136752,2867.0,JMF152,237.60,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,...,0.0,0.000000,0.000000,4514.40,3,-3.0,NaN,NaN,NaN,NaN
6,AA136752,2867.0,JMF153,475.20,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,...,0.0,0.000000,0.000000,4514.40,3,-3.0,NaN,NaN,NaN,NaN
7,AA136752,2867.0,JMF154,356.40,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,...,0.0,0.000000,0.000000,4514.40,3,-3.0,NaN,NaN,NaN,NaN
8,AA136752,2867.0,JMF155,23.76,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,...,0.0,0.000000,0.000000,4514.40,3,-3.0,NaN,NaN,NaN,NaN
9,AA136752,2885.0,JMF164,25.87,2019-05-27,20.0,e-Direct,Category_Tower_07,EA,N,...,0.0,0.003200,-0.001600,12418.56,3,-3.0,NaN,NaN,NaN,NaN


In [58]:
pvcc_margin.columns

Index(['CONTRACT_ID', 'PG_ID', 'NPC', 'B1_SELL_PRICE', 'EFFECTIVE_DATE',
       'PERCENTAGE_RATE', 'CHANNEL_x', 'ITEM_CLASS', 'PRIMARY_UOM_CODE',
       'MBMS', 'B1_EXVAT', 'B1_National_BUY_PRICE', 'CP_EFFECTIVE_DATE',
       '%Margin', 'PG_NAME', 'Category', 'CENTRALLY_FUNDED', 'CHANNEL_y',
       'nat_median', 'st_dev', 'Mean', 'YTD_LIBRA', 'Channel_Margin',
       'Margin Median Difference', 'SUPPLY_ROUTE_x', 'PERCENTAGE_MARGIN_x',
       'SUPPLY_ROUTE_y', 'PERCENTAGE_MARGIN_y'],
      dtype='object')

In [59]:
T7_outliers_with_margin= pvcc_margin.drop(['NPC', 'B1_SELL_PRICE', 'EFFECTIVE_DATE',
       'PERCENTAGE_RATE',  'ITEM_CLASS', 'PRIMARY_UOM_CODE',
       'MBMS', 'B1_EXVAT', 'CP_EFFECTIVE_DATE',
       'PG_NAME', 'Category', 'CENTRALLY_FUNDED', 'YTD_LIBRA'], axis=1)
T7_outliers_with_margin

,CONTRACT_ID,PG_ID,CHANNEL_x,B1_National_BUY_PRICE,%Margin,CHANNEL_y,nat_median,st_dev,Mean,Channel_Margin,Margin Median Difference,SUPPLY_ROUTE_x,PERCENTAGE_MARGIN_x,SUPPLY_ROUTE_y,PERCENTAGE_MARGIN_y
0,AA136744,3449.0,e-Direct,146.25,0.000,e-Direct,0.0,0.000000,0.000000,3,-3.0,NaN,NaN,NaN,NaN
1,AA136744,3449.0,e-Direct,224.25,0.000,e-Direct,0.0,0.000000,0.000000,3,-3.0,NaN,NaN,NaN,NaN
2,AA136752,2867.0,e-Direct,19.50,0.000,e-Direct,0.0,0.000000,0.000000,3,-3.0,NaN,NaN,NaN,NaN
3,AA136752,2867.0,e-Direct,198.00,0.000,e-Direct,0.0,0.000000,0.000000,3,-3.0,NaN,NaN,NaN,NaN
4,AA136752,2867.0,e-Direct,297.00,0.000,e-Direct,0.0,0.000000,0.000000,3,-3.0,NaN,NaN,NaN,NaN
5,AA136752,2867.0,e-Direct,198.00,0.000,e-Direct,0.0,0.000000,0.000000,3,-3.0,NaN,NaN,NaN,NaN
6,AA136752,2867.0,e-Direct,396.00,0.000,e-Direct,0.0,0.000000,0.000000,3,-3.0,NaN,NaN,NaN,NaN
7,AA136752,2867.0,e-Direct,297.00,0.000,e-Direct,0.0,0.000000,0.000000,3,-3.0,NaN,NaN,NaN,NaN
8,AA136752,2867.0,e-Direct,19.80,0.000,e-Direct,0.0,0.000000,0.000000,3,-3.0,NaN,NaN,NaN,NaN
9,AA136752,2885.0,e-Direct,21.56,-0.008,e-Direct,0.0,0.003200,-0.001600,3,-3.0,NaN,NaN,NaN,NaN


In [60]:
T7_outliers_with_margin.to_excel(output_filepath+r'/T7_outliers_with_margin_'+curr_date+'.xlsx', index = False) 

## Part 5 - Drill into PGIDs with unusual stats

## Margin outliers

In [61]:
one_pgid=4851
one_contract='AA133964'

PGID_Contract=df[(df['PG_ID']==one_pgid) & (df['CONTRACT_ID']==one_contract)]
#df.loc[df['CONTRACT_ID']>0.01
PGID_Contract

,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID,%Margin


In [62]:
national_margin_outliers=df.sort_values(by=['%Margin'],ascending=False)
national_margin_outliers

,NPC,PG_ID,B1_SELL_PRICE,EFFECTIVE_DATE,PERCENTAGE_RATE,CHANNEL,ITEM_CLASS,PRIMARY_UOM_CODE,MBMS,B1_EXVAT,B1_National_BUY_PRICE,CP_EFFECTIVE_DATE,CONTRACT_ID,%Margin
1343,FBE15668,1918.0,1490.83,2023-03-02,20.0,e-Direct,Category_Tower_07,EA,N,1242.358333,317.6280,2022-01-04,AA136756,74.433
294,JAH020,6606.0,652.71,2022-07-28,20.0,e-Direct,Category_Tower_07,BX,N,543.925000,271.9632,2022-01-04,AA136734,50.000
295,JAH022,6606.0,481.79,2022-07-28,20.0,e-Direct,Category_Tower_07,BX,N,401.491667,200.7489,2022-01-04,AA136734,49.999
1178,JAH022,4375.0,334.58,2022-01-04,20.0,e-Direct,Category_Tower_07,BX,N,278.816667,200.7489,2022-01-04,AA136734,28.000
310,JAH020,4375.0,453.27,2022-01-04,20.0,e-Direct,Category_Tower_07,BX,N,377.725000,271.9632,2022-01-04,AA136734,28.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,JAH418,4370.0,97.14,2021-11-29,20.0,e-Direct,Category_Tower_07,BX,N,80.950000,81.8800,2022-01-04,AA136734,-1.149
1333,FBE18195,5571.0,14.45,2022-11-02,20.0,e-Direct,Category_Tower_07,EA,N,12.041667,13.0000,2022-01-04,AA136762,-7.958
416,JMF127,2722.0,19.20,2019-03-29,20.0,e-Direct,Category_Tower_07,EA,N,16.000000,17.5900,2022-03-05,AA137668,-9.938
1332,JAH199,5571.0,251.76,2021-11-29,20.0,e-Direct,Category_Tower_07,BX,N,209.800000,270.0000,2022-01-04,AA136736,-28.694


In [63]:
margin_median_top100 = national_margin_outliers.nlargest(100,'%Margin')
margin_median_bottom100 = national_margin_outliers.nsmallest(100,'%Margin')

In [64]:
# create a excel writer object
with pd.ExcelWriter("T7_margin_outliers_with_libra.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    margin_median_top100.to_excel(writer, sheet_name="Top100", index=False)
    margin_median_bottom100.to_excel(writer, sheet_name="Bottom100", index=False)
  

In [65]:
T7_PGID_margin_stats.to_excel('T7_PGID_margin_stats_'+curr_date+'.xlsx', index = False) 

NameError: name 'T7_PGID_margin_stats' is not defined

## Check Sales

In [ ]:
ytd_sales = pd.read_csv("libra2023_YTD_Sales.csv", header = 0)

In [ ]:
one_pgid=ytd_sales.loc[ytd_sales['PG_ID']==5837]
print(np.shape(one_pgid))
one_pgid

In [ ]:
libra_df=pd.read_csv('Libra2023YTD_20230203.csv')
libra_df.head()

In [ ]:
libra_df.groupby('NHS_PERIOD_NO').size().sort_values(ascending=False) \
  .reset_index(name='NPC')

In [ ]:
one_pgid_libra=libra_df.loc[libra_df['PG_ID']==5837]
one_pgid_libra.sort_values(by=['NPC'],ascending=False)

In [ ]:
one_pgid_libra.to_excel('T7_libra_5837_'+curr_date+'.xlsx', index = False) 

## Merge top 100, bottom 100 with YTD Libra

In [ ]:
margin_median_top100

In [ ]:
top100_with_libra= pd.merge(margin_median_top100, libra_df,  how='left', on=['PG_ID','NPC'])
top100_with_libra

In [ ]:
bottom100_with_libra= pd.merge(margin_median_bottom100, libra_df,  how='left', on=['PG_ID','NPC'])
bottom100_with_libra

In [ ]:
# create a excel writer object
with pd.ExcelWriter("T7_margin_outliers_with_libra.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    top100_with_libra.to_excel(writer, sheet_name="Top100", index=False)
    bottom100_with_libra.to_excel(writer, sheet_name="Bottom100", index=False)

## Sand-pit

In [ ]:
import plotly.express as px
df1 = px.data.tips()
fig = px.histogram(df1, x="total_bill", nbins=20)
fig.show()

In [ ]:
national_buy_zero = df.loc[df['%Margin'].isna()]
np.shape(national_buy_zero)

In [ ]:
national_buy_zero = df.loc[df['%Margin'].isna()]

In [ ]:
df.sort_values(by=['B1_SELL_PRICE'],ascending=True)